# CNN on Cats vs Dogs Dataset
Completed Assignment Notebook with Final Task

In [ ]:

import os
import random
from shutil import copyfile
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint as MCP, EarlyStopping as ES
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt


## Task 1: Data Generators

In [ ]:

datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

train_set = datagen.flow_from_directory(
    directory=TRAIN,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

test_set = datagen.flow_from_directory(
    directory=TEST,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)


## Task 2: Build CNN Model

In [ ]:

model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(256,256,3)),
    keras.layers.MaxPooling2D(pool_size=(2,2), strides=2),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2), strides=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dense(units=1, activation='sigmoid')
])

model.summary()


## Task 3: Compile Model

In [ ]:

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


### Callbacks

In [ ]:

early_stop= ES(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    restore_best_weights=True,
    verbose=1,
    mode='auto'
)

checkpoint= MCP(
    filepath='best_model.weights.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True ,
    verbose=1,
    mode="auto"
)


## Task 4: Train Model

In [ ]:

track = model.fit(
    train_set,
    validation_data=test_set,
    epochs=15,
    callbacks=[early_stop, checkpoint]
)

print(track.history)


## Task 5: Predictions & Evaluation

In [ ]:

predictions_probabilities = model.predict(test_set)
predicted_labels = (predictions_probabilities > 0.5).astype(int).flatten()
true_labels = test_set.classes

# Confusion Matrix
cm = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:\n", cm)

# Classification Report
report = classification_report(true_labels, predicted_labels, target_names=['Cat','Dog'])
print("Classification Report:\n", report)

# Plot Loss Curves
plt.plot(track.history['loss'])
plt.plot(track.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train','Validation'])
plt.show()



## Task 6: Reflection

1. **Final Performance**  
   - Example: Final accuracy ≈ 85–90%, Loss ≈ 0.4–0.5 (depends on run).  

2. **Confusion Matrix**  
   - The diagonal values represent correct predictions. If the model misclassifies cats as dogs more often, it means cat features were harder to learn.  

3. **Training Curves**  
   - If validation loss flattens earlier than training loss, the model risks overfitting. With EarlyStopping, training stops at the optimal point.  

4. **Potential Improvements**  
   - Add dropout layers to reduce overfitting.  
   - Use pre-trained models like VGG16 (Transfer Learning).  
   - Increase training data or use stronger augmentation.  



## Final Task: Conceptual Q/A

**Q1. What are the advantages of Convolutional Layers over Fully Connected Layers?**  
✔ Convolutional layers capture spatial features with fewer parameters, making training faster and more efficient. They are translation-invariant, meaning they can detect patterns regardless of position.  

**Q2. What is the role of Pooling Layers in reducing complexity of a CNN?**  
✔ Pooling layers reduce the spatial size of feature maps, lowering computation and memory requirements. They also make the network robust to small image distortions and translations.  

**Q3. Draw the Comparison of Pooling Layers.**  
- **Max Pooling**: Selects the maximum value in each region. Good for sharp features.  
- **Average Pooling**: Averages values in each region. Produces smoother outputs but less detail.  
- **Global Average Pooling**: Averages across the entire feature map, often used before fully connected layers in modern CNNs.  
